<a href="https://colab.research.google.com/github/t1seo/AIFFEL/blob/master/FUNDAMENTALS/10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10. 🦄 가랏, 몬스터볼! 전설의 포켓몬 찾아 삼만리

## 10-1. 들어가며
- 다양한 피쳐가 있는 데이터셋을 밑바닥부터 샅샅이 뜯어보고, 전설의 포켓몬을 분류하기 위한 피쳐에는 무엇이 있는지 생각해 본다.
- 모델 학습을 시작하기 전 모든 컬럼에 대해 그래프 시각화, 피벗 테이블 등을 활용하며 다양한 방법으로 충분한 EDA를 진행한다.
- 모델 학습에 넣기 위해서 전처리가 필요한 범주형/문자열 데이터에 대한 전처리를 원-핫 인코딩 등으로 적절하게 진행한다.
- 전체 데이터셋을 train/test 데이터셋으로 나누고, 적절한 분류 모델(Decision Tree)을 선택해 학습시키며 베이스라인과 비교해본다.

## 10-2. 안녕, 포켓몬과 인사해!
포켓몬의 이름, 속성, 또는 공격력이나 방어력 등과 같은 스탯 값만을 가지고 전설의 포켓몬인지 아닌지를 구별해낼 수 있을까요?

오늘은 이러한 분류 문제를 풀기 위해 데이터를 밑바닥부터 샅샅이 뜯어보는 연습을 할 것입니다.
이러한 과정을 탐색적 **데이터 분석(Exploratory Data Analysis, 이하 EDA)**이라고 합니다.
EDA는 더 좋은 데이터 분석과 더 좋은 머신러닝 모델을 만들기 위해 필수적인 과정입니다.

더욱 의미 있는, 그리고 더 나은 성능의 모델을 만들어내기 위해서는 무엇을 고민해야 하는지, 그 흐름을 한번 따라와 보시길 바랍니다!
이러한 흐름에 익숙해진다면 어떤 데이터셋을 만나더라도 충분히 빠르고 치밀하게 데이터셋을 다룰 수 있게 될 것입니다.

## 10-3. 포켓몬, 그 데이터는 어디서 구할까?
- [[캐글] Pokemon with stats](https://www.kaggle.com/abcsds/pokemon)

This data set includes 721 Pokemon, including their **number, name, first and second type, and basic stats: HP, Attack, Defense, Special Attack, Special Defense, and Speed**. It has been of great use when teaching statistics to kids. With certain types you can also give a geeky introduction to machine learning.


*총 11개의 피쳐가 있으며, 각각은 포켓몬의 ID, 이름, 첫 번째 속성, 두 번째 속성, 스탯의 총합, HP, 공격력, 방어력, 특수 공격력, 특수 방어력, 그리고 속도 이다.*


This are the raw attributes that are used for calculating how much damage an attack will do in the games. This dataset is about the pokemon games (NOT pokemon cards or Pokemon Go).

The data as described by Myles O'Neill is:

- #: ID for each pokemon
- Name: Name of each pokemon
- Type 1: Each pokemon has a type, this determines weakness/resistance to attacks
- Type 2: Some pokemon are dual type and have 2
- Total: sum of all stats that come after this, a general guide to how strong a pokemon is
- HP: hit points, or health, defines how much damage a pokemon can withstand before fainting
- Attack: the base modifier for normal attacks (eg. Scratch, Punch)
- Defense: the base damage resistance against normal attacks
- SP Atk: special attack, the base modifier for special attacks (e.g. fire blast, bubble beam)
- SP Def: the base damage resistance against special attacks
- Speed: determines which pokemon attacks first each round


The data for this table has been acquired from several different sites, including:

- [pokemon.com](https://www.pokemon.com/us/pokedex/)
- [pokemondb](https://pokemondb.net/pokedex)
- [bulbapedia](https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_National_Pok%C3%A9dex_number)

One question has been answered with this database: The type of a pokemon cannot be inferred only by it's Attack and Deffence. It would be worthy to find which two variables can define the type of a pokemon, if any. Two variables can be plotted in a 2D space, and used as an example for machine learning. This could mean the creation of a visual example any geeky Machine Learning class would love.

In [ ]:
from google.colab import files
file_upload = files.upload()

In [ ]:
pokemon_data = 'Pokemon.csv'

## 10-4. 포켓몬 데이터 불러오기

### 라이브러리 가져오기

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline # 브라우저 내부(inline)에서 바로 그래프 그림 보여지게 한다
%config InlineBackend.figure_format = 'retina' # %matplotlib 뒤에 넣어주면 그래프를 더 높은 해상도로 출력한다.

### 데이터셋을 pandas로 불러오기

In [ ]:
# 연습을 위해 원본을 따로 만들어둔다
original_data = pd.read_csv(pokemon_data)

In [ ]:
pokemon = original_data.copy() # original로부터 deep copy
print(pokemon.shape)

In [ ]:
pokemon.head()

- 데이터셋은 총 800행, 13열로 이루어져 있다.
- 포켓몬은 총 800마리이고, 각 포켓몬을 설명하는 특성(feature)는 13개이다.

타겟으로 두고 확인할 데이터는 `Legendary (전설의 포켓몬인지 아닌지의 여부)`이므로 `Legendary == True` 값을 가지는 레전드 포켓몬 데이터셋은 `legendary` 변수에, `Legendary == False` 값을 가지는 일반 포켓몬 데이터셋은 `ordinary` 변수에 저장해 둔다.



In [ ]:
# 전설의 포켓몬 데이터셋
legendary = pokemon[pokemon["Legendary"] == True].reset_index(drop=True)
print(legendary.shape)
legendary.head()

- 전설의 포켓은은 65마리가 있다.

---

- `Legendary`가 `True`인 데이터만 따로 떼어내서 저장했다.
- `reset_index`는  인덱스를 다시 처음부터 재배열 해주는데 `drop=True`를 옵션으로 지정해주면 기존 인덱스는 버리고 재배열 해준다. 이 옵션을 지정해주지 않으면 기존 인덱스는 컬럼에 들어가버린다.

In [ ]:
# 일반 포켓몬 데이터셋
ordinary = pokemon[pokemon["Legendary"] == False].reset_index(drop=True)
print(ordinary.shape)
ordinary.head()

## 10-5. 전설의 포켓몬? 먼저 샅샅이 살펴보자! (1) 결측치와 전체 칼럼


### 빈 데이터 확인하기

In [ ]:
# 컬럼별 결측치 확인
pokemon.isnull().sum()

- `Type 2` 컬럼에만 총 386개의 결측치가 있다. 두 번째 속성이 없는 포켓몬이 있는 것 같다.

데이터셋을 다룰 때 빈 데이터를 다루는 것은 매우 조심스러운 일입니다. 데이터셋의 성격에 따라 빈 데이터를 어떻게 다루어야 할지에 대한 방법이 달라지기 때문이죠.

## 전체 컬럼 이해하기
데이터가 800개이고, 컬럼이 13개로 큰 데이터셋이 아니므로, 최대한 모든 데이터들을 하나하나 이해해 볼 수 있겠습니다.

In [ ]:
# 컬럼 개수 출력
print(len(pokemon.columns))

In [ ]:
# 컬럼 이름 출력
pokemon.columns

- **#** : 포켓몬 Id number. 같은 포켓몬이지만 성별이 다른 경우 등은 같은 #값을 가진다. `int`
- **Name** : 포켓몬 이름. 포켓몬 각각의 이름으로, 이름 데이터는 800개의 포켓몬이 모두 다르다. (unique) `str`
- **Type 1** : 첫 번째 속성. 속성을 하나만 가지는 경우 Type 1에 입력된다. `str`
- **Type 2** : 두 번째 속성. 속성을 하나만 가지는 포켓몬의 경우 Type 2는 NaN(결측값)을 가진다. `str`
- **Total** : 전체 6가지 스탯의 총합. `int`
- **HP** : 포켓몬의 체력. `int`
- **Attack** : 물리 공격력. (scratch, punch 등) `int`
- **Defense** : 물리 공격에 대한 방어력. `int`
- **Sp. Atk** : 특수 공격력. (fire blast, bubble beam 등) `int`
- **Sp. Def** : 특수 공격에 대한 방어력. `int`
- **Speed** : 포켓몬 매치에 대해 어떤 포켓몬이 먼저 공격할지를 결정. (더 높은 포켓몬이 먼저 공격한다) `int`
- **Generation** : 포켓몬의 세대. 현재 데이터에는 6세대까지 있다. `int`
- **Legendary** : 전설의 포켓몬 여부. !! **Target feature** !! `bool`

## 10-6. 전설의 포켓몬? 먼저 샅샅이 살펴보자! (2) ID와 이름

### # : ID number


In [ ]:
# 총 몇 종류의 # 값이 있는지 확인
len(set(pokemon['#']))  # set 사용

- 전체 데이터는 총 800개 인데 `#`을 집합으로 만든 자료형은 그보다 작은 721개의 데이터를 가지고 있다.

In [ ]:
pokemon[pokemon['#'] == 6]

- 예를 들어 #6의 포켓몬은 Charizard, CharizardMega Charizard X, CharizardMega Charizard Y 세 개로 나뉩니다.
- 기본 포켓몬인 Charizard로부터 시작해서 진화한 Mega Charizard가 있고, X, Y는 성별을 나타내는 것으로 보입니다.



In [ ]:
# 유일한 이름의 개수 확인
len(set(pokemon['Name']))

- 모든 이름들은 unique 하다.

## 10-7. 전설의 포켓몬? 먼저 샅샅이 살펴보자! (3) 포켓몬의 속성

### Type 1 & Type 2 : 포켓몬의 속성

In [ ]:
# 무작위로 두 마리의 포켓몬 살펴보기
pokemon.loc[[6, 10]]

- 6번 포켓몬인 Charizard는 Fire와 Flying 속성 두 가지를, 8번 포켓몬인 Wartortle은 Water 속성 단 한 가지만 가진다.
- 속성은 기본적으로 하나, 또는 최대 두개까지 가질 수 있는 것을 알 수 있다.

In [ ]:
#  각 속성의 종류는 총 몇 가지인지 알아본다
print("Type 1: ", len(list(set(pokemon["Type 1"]))))
print("Type 2: ", len(list(set(pokemon["Type 2"]))))

- Type 2의 속성이 하나 더 많다.

In [ ]:
# 각자의 집합을 차집합해준다
set(pokemon["Type 2"]) - set(pokemon["Type 1"])

- 둘의 차집합은 `NaN`으로 `Type 1`과 `Type 2`에는 모두 같은 세트의 데이터가 들어가 있는 것을 확인할 수 있다.

In [ ]:
# 포켓몬들의 모든 타입을 types 변수에 저장
types = list(set(pokemon["Type 1"]))
print(len(types))
print(types)

In [ ]:
# 데이터가 비어있는 NaN의 개수 확인
pokemon["Type 2"].isna().sum() # pandas isna() 함수 사용

- 총 386개의 포켓몬은 속성을 하나만 가지고 나머지는 두 개의 속성을 가진다.
---
- 참고: [판다스 isna() 함수](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isna.html)
